In [1]:
#1.Web Scraping

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_books():
    url = "https://books.toscrape.com/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    books = soup.find_all("article", class_="product_pod")
    data = []

    for book in books:
        title = book.h3.a["title"]
        price = book.find("p", class_="price_color").text.strip()
        availability = book.find("p", class_="instock availability").text.strip()

        data.append({
            "Title": title,
            "Price": price,
            "Availability": availability
        })

    # Save to CSV
    df = pd.DataFrame(data)
    df.to_csv("books.csv", index=False, encoding='utf-8-sig')
    print("✅ Book data saved to 'books.csv'.")

scrape_books()

✅ Book data saved to 'books.csv'.


In [3]:
# 2. Hangman Game

In [4]:
import random

def play_hangman():
    easy = [
        {"word": "cat", "hint": "A common pet"},
        {"word": "car", "hint": "Has wheels and engine"},
        {"word": "sun", "hint": "Bright and hot"}
    ]
    medium = [
        {"word": "guitar", "hint": "A musical instrument"},
        {"word": "school", "hint": "Place for learning"},
        {"word": "planet", "hint": "Orbits a star"}
    ]
    hard = [
        {"word": "python", "hint": "A programming language"},
        {"word": "elephant", "hint": "Large land animal"},
        {"word": "dinosaur", "hint": "Extinct reptile"}
    ]

    print("Welcome to Hangman!")
    level = input("Choose difficulty (easy / medium / hard): ").lower()

    if level == "easy":
        words = easy
    elif level == "medium":
        words = medium
    elif level == "hard":
        words = hard
    else:
        print("Invalid input. Defaulting to medium.")
        words = medium

    chosen = random.choice(words)
    word = chosen["word"]
    hint = chosen["hint"]
    guessed = ["_"] * len(word)
    attempts = 6
    used_letters = []

    print("Hint:", hint)

    while attempts > 0 and "_" in guessed:
        print("\nWord:", " ".join(guessed))
        print("Attempts left:", attempts)
        print("Used letters:", ", ".join(used_letters))
        guess = input("Guess a letter: ").lower()

        if not guess or len(guess) != 1 or not guess.isalpha():
            print("Enter a single alphabet only.")
            continue

        if guess in used_letters:
            print("Already guessed.")
            continue

        used_letters.append(guess)

        if guess in word:
            for i, char in enumerate(word):
                if char == guess:
                    guessed[i] = guess
            print("✅ Correct!")
        else:
            attempts -= 1
            print("❌ Wrong!")

    print("\nFinal word:", word)
    if "_" not in guessed:
        print("🎉 You won!")
    else:
        print("😢 You lost!")

play_hangman()

Welcome to Hangman!


Choose difficulty (easy / medium / hard):  easy


Hint: A common pet

Word: _ _ _
Attempts left: 6
Used letters: 


Guess a letter:  g


❌ Wrong!

Word: _ _ _
Attempts left: 5
Used letters: g


...

✅ Correct!

Word: c _ _
Attempts left: 5
Used letters: g, c


Guess a letter:  a


✅ Correct!

Word: c a _
Attempts left: 5
Used letters: g, c, a


Guess a letter:  t


✅ Correct!

Final word: cat
🎉 You won!


In [5]:
# Hangman Game (Alternative)

In [6]:
import tkinter as tk
import random

words = {
    "easy": [("apple", "A fruit", "Fruits"), ("cat", "A pet animal", "Animals"), ("sun", "It shines", "Nature")],
    "medium": [("python", "Programming language", "Technology"), ("guitar", "Musical instrument", "Music"), ("planet", "Part of solar system", "Space")],
    "hard": [("microscope", "Used in laboratories", "Science"), ("philosophy", "Study of knowledge", "Subjects"), ("democracy", "Form of government", "Politics")]
}

chosen_word = ""
display_word = []
wrong_guesses = 0
max_wrong = 6
guessed_letters = []
hint_text = ""
category_text = ""

root = tk.Tk()
root.title("Hangman Game - Improved")
root.geometry("600x600")
root.resizable(False, False)

canvas = tk.Canvas(root, width=400, height=300, bg="white")
canvas.pack(pady=10)

# UI labels
category_label = tk.Label(root, text="", font=("Arial", 14, "bold"))
category_label.pack()

word_label = tk.Label(root, text="", font=("Arial", 20))
word_label.pack(pady=5)

hint_label = tk.Label(root, text="", font=("Arial", 12))
hint_label.pack()

status_label = tk.Label(root, text="", font=("Arial", 14, "bold"))
status_label.pack(pady=10)

entry_frame = tk.Frame(root)
entry_frame.pack()

entry = tk.Entry(entry_frame, font=("Arial", 16), width=5, justify='center')
entry.grid(row=0, column=0, padx=5)
entry.bind("<Return>", lambda event: check_guess(entry.get().lower()))  # Enter key handling


submit_btn = tk.Button(entry_frame, text="Submit", font=("Arial", 12), command=lambda: check_guess(entry.get().lower()))
submit_btn.grid(row=0, column=1, padx=5)

reset_btn = tk.Button(root, text="Play Again", font=("Arial", 12), command=lambda: start_game(difficulty_var.get()))
reset_btn.pack(pady=5)

difficulty_var = tk.StringVar(value="easy")
difficulty_menu = tk.OptionMenu(root, difficulty_var, "easy", "medium", "hard")
difficulty_menu.pack()

# --- Drawing ---

def draw_base():
    canvas.delete("all")
    canvas.create_line(50, 250, 150, 250)     # Base
    canvas.create_line(100, 250, 100, 50)     # Vertical
    canvas.create_line(100, 50, 200, 50)      # Top bar
    canvas.create_line(200, 50, 200, 70, tags="rope")   # Rope

def draw_stickman_part(wrong):
    parts = {
        1: canvas.create_oval(180, 70, 220, 110, width=2, tags="head"),
        2: canvas.create_line(200, 110, 200, 170, width=2, tags="body"),
        3: canvas.create_line(200, 130, 170, 150, width=2, tags="left_arm"),
        4: canvas.create_line(200, 130, 230, 150, width=2, tags="right_arm"),
        5: canvas.create_line(200, 170, 170, 210, width=2, tags="left_leg"),
        6: canvas.create_line(200, 170, 230, 210, width=2, tags="right_leg")
    }

def stickman_safe_slide():
    parts = ["head", "body", "left_arm", "right_arm", "left_leg", "right_leg"]
    target_bottom = 250  # Final Y where bottom of stickman (legs) should align

    def slide():
        all_coords = [canvas.coords(part) for part in parts if canvas.coords(part)]
        current_bottom = max((coord[3] if len(coord) > 3 else coord[1]) for coord in all_coords)

        if current_bottom < target_bottom:
            for part in parts:
                if canvas.coords(part):
                    canvas.move(part, 0, 3)  # smooth but slower
            root.after(40, slide)
        else:
            canvas.create_text(200, 260, text="🎉 Landed Safely!", font=("Arial", 14), fill="green")
            status_label.config(text="✅ You Won!", fg="green")

    slide()


# --- Logic ---

def start_game(difficulty):
    global chosen_word, display_word, wrong_guesses, guessed_letters, hint_text, category_text
    draw_base()
    wrong_guesses = 0
    guessed_letters = []
    
    word, hint_text, category_text = random.choice(words[difficulty])
    chosen_word = word
    display_word = ["_" for _ in chosen_word]

    word_label.config(text=" ".join(display_word))
    hint_label.config(text=f"Hint: {hint_text}")
    category_label.config(text=f"Category: {category_text}")
    status_label.config(text="", fg="black")
    entry.delete(0, tk.END)

def check_guess(letter):
    global wrong_guesses
    entry.delete(0, tk.END)

    if not letter.isalpha() or len(letter) != 1:
        status_label.config(text="Enter a single letter.", fg="red")
        return

    if letter in guessed_letters:
        status_label.config(text="Already guessed!", fg="orange")
        return

    guessed_letters.append(letter)

    if letter in chosen_word:
        for idx, char in enumerate(chosen_word):
            if char == letter:
                display_word[idx] = letter
        word_label.config(text=" ".join(display_word))
        if "_" not in display_word:
            stickman_safe_slide()
    else:
        wrong_guesses += 1
        draw_stickman_part(wrong_guesses)
        if wrong_guesses >= max_wrong:
            word_label.config(text=" ".join(chosen_word))
            status_label.config(text="❌ You Lost!", fg="red")

# Start game
start_game(difficulty_var.get())
root.mainloop()